In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
    
class MNIST_data:
    def __init__(self, is_train_dataset, need_shuffle):
        all_data = []
        all_labels = []

        if is_train_dataset:
            all_data = mnist.train.images
            all_labels = mnist.train.labels
        else:
            all_data = mnist.test.images
            all_labels = mnist.test.labels

        self._data = np.vstack(all_data)              
        self._labels = np.vstack(all_labels)
        print(self._data.shape)

        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:          
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is lager than all example")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_data = MNIST_data(True, True)
test_data = MNIST_data(False, False)

ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

In [2]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

w = tf.get_variable('w', [x.get_shape()[-1], 10], initializer=tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [10], initializer=tf.constant_initializer(0.0))

y_ = tf.matmul(x, w) + b
p_y = tf.nn.softmax(y_)

loss = tf.reduce_mean(tf.square(y - p_y))

predict = tf.argmax(y_, 1)
predict_one_hot = tf.one_hot(predict, 10, dtype=tf.float32)
correct_predict = tf.equal(predict_one_hot, y)
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

with tf.name_scope('train_scope'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [3]:
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = 'run_mnist_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)

if not os.path.exists(run_dir):
    os.mkdir(run_dir)

train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [4]:
init = tf.global_variables_initializer()

batch_size = 20
train_steps = 10000
test_steps = 100
output_summary_every_steps = 100

with tf.Session() as sess:
    sess.run(init)

    train_writer = tf.summary.FileWriter(train_log_dir)
    test_writer = tf.summary.FileWriter(test_log_dir)

    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)

    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)

        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i + 1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)

        eval_ops_results = sess.run([loss, accuracy, train_op], feed_dict = {x: batch_data, y: batch_labels})

        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test], feed_dict={x: fixed_test_batch_data, y: fixed_test_batch_labels})[0]
            test_writer.add_summary(test_summary_str, i+1)

        loss_val, acc_val = eval_ops_results[0:2]

        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 0.12080, acc: 0.87000
[Train] Step: 1000, loss: 0.12014, acc: 0.86000
[Train] Step: 1500, loss: 0.09186, acc: 0.89000
[Train] Step: 2000, loss: 0.08507, acc: 0.90000
[Train] Step: 2500, loss: 0.09933, acc: 0.89000
[Train] Step: 3000, loss: 0.09733, acc: 0.88000
[Train] Step: 3500, loss: 0.06419, acc: 0.93000
[Train] Step: 4000, loss: 0.05684, acc: 0.94000
[Train] Step: 4500, loss: 0.07134, acc: 0.92000
[Train] Step: 5000, loss: 0.04989, acc: 0.95000
[Test ] Step: 5000, acc: 0.94090
[Train] Step: 5500, loss: 0.03583, acc: 0.94000
[Train] Step: 6000, loss: 0.02289, acc: 0.97000
[Train] Step: 6500, loss: 0.03897, acc: 0.95000
[Train] Step: 7000, loss: 0.02754, acc: 0.97000
[Train] Step: 7500, loss: 0.01948, acc: 0.98000
[Train] Step: 8000, loss: 0.04289, acc: 0.95000
[Train] Step: 8500, loss: 0.01539, acc: 0.98000
[Train] Step: 9000, loss: 0.03988, acc: 0.94000
[Train] Step: 9500, loss: 0.02174, acc: 0.97000
[Train] Step: 10000, loss: 0.02413, acc: 0.97000
[Test ]

In [ ]:
# 0.9692